In [1]:
from brian2 import*
%matplotlib inline
runtime = 30*ms

In [ ]:
#run this only if previous cell gives an error else skip
clear_cache('cython')

In [2]:
import numpy as np
with open('try.txt', "r") as file:
    # Read each line
    arrays_list = []
    for line in file:
        # Split the line by comma and convert to float
        numbers = [float(num) for num in line.strip().split(",")]

        # Convert the first 64 numbers into a NumPy array
        array_64 = np.array(numbers[:64])

        # Append the array to the list
        arrays_list.append(array_64)

# Convert the list of arrays into a 2D NumPy array
arrays_2d = np.array(arrays_list)
for i in range(len(arrays_2d)):
    for j in range(64):
        if(arrays_2d[i][j]<7):
            arrays_2d[i][j]= 0.01
        else:
            arrays_2d[i][j]=0.1

In [3]:

import numpy as np
import pandas as pd
wts = []
df = pd.read_csv('weights.csv', header=None)
wts = df.values
k = 100
wts=wts.T
#upload weights in SNN
wts_snn =[]
for i in range(0,64):
        for j in range(0,10):
            wts_snn.append((wts[i][j])/k)



In [4]:

result = []
for i in range(len(arrays_2d)):  
    start_scope()
    eqs = '''
    dv/dt = (inp-v)/tau : 1
    inp : 1
    tau : second
    '''
    #define input and output neuron groups
    I = NeuronGroup(64, eqs, threshold='v>0.008', reset='v = 0', method='exact')
    O = NeuronGroup(10, eqs, threshold='v>0.008', reset='v = 0', method='exact')
    O0 = O[0]
    O1 = O[1]
    O2 = O[2]
    O3 = O[3]
    O4 = O[4]
    O5 = O[5]
    O6 = O[6]
    O7 = O[7]
    O8 = O[8]
    O9 = O[9]

    I.tau = 6*ms
    O.tau = 6*ms
    O.inp = [0,0,0,0,0,0,0,0,0,0]

    #define synapses
    S = Synapses(I, O, 'wt:1', on_pre='v_post += wt')
    S.connect()
    S.wt = wts_snn
    #give input to the neurons from test cases
    I.inp = arrays_2d[i]
    
    #monitor the membrane potential and spiking behaviour of output neurons
    M_in = StateMonitor(I, 'v', record=True)
    M_out = StateMonitor(O, 'v', record=True)
    SpkO0 = SpikeMonitor(O[0])
    SpkO1= SpikeMonitor(O[1])
    SpkO2= SpikeMonitor(O[2])
    SpkO3= SpikeMonitor(O[3])
    SpkO4= SpikeMonitor(O[4])
    SpkO5= SpikeMonitor(O[5])
    SpkO6= SpikeMonitor(O[6])
    SpkO7= SpikeMonitor(O[7])
    SpkO8= SpikeMonitor(O[8])
    SpkO9= SpikeMonitor(O[9])
  
    run(runtime)

    #Monitor spinking time of each output neuron to find the one which spikes first
    spk =[]
    spk.append(min(SpkO0.t[:], default = runtime))
    spk.append(min(SpkO1.t[:], default = runtime))
    spk.append(min(SpkO2.t[:], default = runtime))
    spk.append(min(SpkO3.t[:], default = runtime))
    spk.append(min(SpkO4.t[:], default = runtime))
    spk.append(min(SpkO5.t[:], default = runtime))
    spk.append(min(SpkO6.t[:], default = runtime)) 
    spk.append(min(SpkO7.t[:], default = runtime))   
    spk.append(min(SpkO8.t[:], default = runtime))
    spk.append(min(SpkO9.t[:], default = runtime))
    min_time = min(spk)
    #append the output predicted by SNN in the result array
    result.append(spk.index(min_time))


In [5]:
print("Predicted by SNN:",result)
#read try.txt to print actual digits given as input for comparison with SNN's result
with open('try.txt', "r") as file:
    # Actual digit
    actual = []
    for line in file:
        # Split the line by comma and convert to float
        numb = [int(num) for num in line.strip().split(",")]
        # Convert the first 64 numbers into a NumPy array
        actual.append(numb[64])
print("   Actual digit :",actual)

Predicted by SNN: [0, 1, 1, 3, 4, 9, 1, 7, 8, 9, 0, 1, 2, 3, 4, 5, 1, 7, 8, 9, 0, 1, 1, 3, 1, 5, 6, 7, 8, 5, 0, 1, 5, 5, 1, 5, 0, 1, 8, 9, 8, 4, 1, 7, 7, 3, 1, 1, 0, 0, 2, 1, 7, 1, 1, 0, 1, 1, 6, 3, 3, 7, 2, 3, 4, 6, 6, 6, 1, 1, 1, 5, 0, 9, 5, 2, 8, 1, 0, 0, 1, 7, 6, 3, 1, 1, 7, 1, 1, 3, 1, 2, 9, 1, 7, 1, 1, 1, 2, 1, 4, 0, 5, 1, 6, 3, 1, 1, 7, 5, 4, 4, 7, 2, 8, 1, 2, 5, 7, 9, 5, 1, 1, 1, 4, 3, 0, 8, 9, 2, 0, 1, 1, 3, 1, 5, 6, 7, 8, 9, 0, 1, 2, 2, 1, 5, 6, 7, 8, 9, 0, 1, 2, 3, 1, 5, 1, 7, 1, 1, 0, 1, 5, 5, 6, 5, 0, 8, 8, 9, 1, 1, 9, 7, 7, 3, 5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 1, 6, 3, 2, 1, 3, 3, 1, 6, 6, 6, 1, 3, 1, 5, 0, 9, 5, 2, 1, 1, 0, 0, 1, 1, 6, 2, 1, 8, 7, 3, 1, 3, 9, 1, 7, 6, 1, 4, 3, 1, 1, 0, 5, 3, 6, 9, 6, 1, 7, 5, 1, 4, 2, 2, 1, 1, 1, 5, 5, 1, 1, 1, 1, 9, 0, 1, 1, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 0, 1, 2, 1, 4, 5, 6, 7, 8, 8, 0, 1, 2, 3, 4, 3, 6, 7, 8, 9, 0, 9, 5, 3, 6, 5, 0, 8, 8, 9, 8, 4, 1, 7, 7, 3, 5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3, 3, 7, 3, 3, 4, 6, 6, 6, 4, 9, 1, 

In [ ]:
count=0
#calcualting accuracy
for i in range(len(result)):
    if(result[i]==actual[i]):
        count= count+1
print("accuracy = ", ((count*100)/len(result)))


In [ ]:
#plot the results
%matplotlib inline
out_spk =np.zeros((10,len(result)))
for i in range(len(result)):
    out_spk[result[i]][i] = 1
input_index = np.arange(1,(len(result)+1))

fig1, axs = plt.subplots(5, 2, figsize=(20, 10))

# Plot neuron 0
axs[0, 0].stem(input_index, out_spk[0], label='Neuron 0')
axs[0, 0].set_xlabel('input index')
axs[0, 0].set_ylabel('v')
axs[0, 0].legend()

# Plot neuron 1
axs[0, 1].stem(input_index, out_spk[1], label='Neuron 1')
axs[0, 1].set_xlabel('input index')
axs[0, 1].set_ylabel('v')
axs[0, 1].legend()

# Plot neuron 2
axs[1, 0].stem(input_index, out_spk[2], label='Neuron 2')
axs[1, 0].set_xlabel('input index')
axs[1, 0].set_ylabel('v')
axs[1, 0].legend()

# Plot neuron 3
axs[1, 1].stem(input_index, out_spk[3], label='Neuron 3')
axs[1, 1].set_xlabel('input index')
axs[1, 1].set_ylabel('v')
axs[1, 1].legend()

# Plot neuron 4
axs[2, 0].stem(input_index, out_spk[4], label='Neuron 4')
axs[2, 0].set_xlabel('input index')
axs[2, 0].set_ylabel('v')
axs[2, 0].legend()

# Plot neuron 5
axs[2, 1].stem(input_index, out_spk[5], label='Neuron 5')
axs[2, 1].set_xlabel('input index')
axs[2, 1].set_ylabel('v')
axs[2, 1].legend()

# Plot neuron 6
axs[3, 0].stem(input_index, out_spk[6], label='Neuron 6')
axs[3, 0].set_xlabel('input index')
axs[3, 0].set_ylabel('v')
axs[3, 0].legend()

# Plot neuron 7
axs[3, 1].stem(input_index, out_spk[7], label='Neuron 7')
axs[3, 1].set_xlabel('input index')
axs[3, 1].set_ylabel('v')
axs[3, 1].legend()

# Plot neuron 8
axs[4, 0].stem(input_index, out_spk[8], label='Neuron 8')
axs[4, 0].set_xlabel('input index')
axs[4, 0].set_ylabel('v')
axs[4, 0].legend()

# Plot neuron 9
axs[4, 1].stem(input_index, out_spk[9], label='Neuron 9')
axs[4, 1].set_xlabel('input index')
axs[4, 1].set_ylabel('v')
axs[4, 1].legend()

fig1.suptitle('Output Patterns', fontsize=16)
# Tighten the layout
plt.tight_layout()

# Display the plot
plt.show()
